In [1]:
DATA_NAME = 'coco-indoor' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'gray'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_diagonal_wavelet_indoor_coco.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [ ]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,12961.88,0.22,0.06,0.03,0.00,-0.00,0.00,-0.00
3,0.22,2545.83,-0.02,0.01,-0.00,0.00,-0.00,0.00
4,0.06,-0.02,406.04,0.00,0.00,-0.00,0.00,-0.00
5,0.03,0.01,0.00,57.84,0.00,0.00,0.00,-0.00
6,0.00,-0.00,0.00,0.00,7.73,0.00,0.00,0.00
7,-0.00,0.00,-0.00,0.00,0.00,1.04,-0.00,0.00
8,0.00,-0.00,0.00,0.00,0.00,-0.00,0.16,0.00
9,-0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,0.02


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,0.00004,0.00003,0.00004,0.00000,-0.00000,0.00000,-0.00001
3,0.00004,1.00000,-0.00002,0.00003,-0.00000,0.00001,-0.00002,0.00003
4,0.00003,-0.00002,1.00000,0.00002,0.00001,-0.00004,0.00004,-0.00002
5,0.00004,0.00003,0.00002,1.00000,0.00002,0.00001,0.00001,-0.00002
6,0.00000,-0.00000,0.00001,0.00002,1.00000,0.00002,0.00002,0.00002
7,-0.00000,0.00001,-0.00004,0.00001,0.00002,1.00000,-0.00003,0.00001
8,0.00000,-0.00002,0.00004,0.00001,0.00002,-0.00003,1.00000,0.00002
9,-0.00001,0.00003,-0.00002,-0.00002,0.00002,0.00001,0.00002,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.33380892125814615

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[2.10187165e+07 7.93572463e+05 1.96287054e+04 3.82501414e+02
 6.41535953e+00 1.03355183e-01 1.87318176e-03 1.62460150e-36]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999566,-0.029121,-0.004443,-0.000631,-0.000085,-0.000011,-0.000002,-2.149958e-07
1,0.028986,0.999197,-0.027396,-0.003783,-0.000507,-0.000068,-0.000011,-1.140865e-06
2,0.005213,0.027132,0.999186,-0.029154,-0.003805,-0.000514,-0.000079,-9.426453e-06
3,0.000888,0.004531,0.028889,0.998980,-0.034096,-0.004481,-0.000695,-8.156343e-05
4,0.000148,0.000756,0.004740,0.033725,0.998352,-0.045651,-0.006916,-8.068389e-04
5,0.000026,0.000134,0.000845,0.005914,0.044775,0.995798,-0.079164,-8.914545e-03
6,0.000006,0.000031,0.000195,0.001369,0.010246,0.077192,0.990067,-1.170515e-01
7,0.000001,0.000007,0.000046,0.000324,0.002418,0.018000,0.115979,9.930855e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.00043417928556932495,
 0.0008029780804044151,
 0.000814372970592836,
 0.0010200228683626689,
 0.001647818539129542,
 0.0042022901686635805,
 0.009932587894323941,
 0.006914498195729313]